# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data=pd.read_csv('../output_data/clean_cities.csv')
city_data

,Unnamed: 0,City Name,Country,Date,Temperature,Latitude,Longitude,Humidity,Cloudiness,Wind Speed
0,0,Kargasok,RU,1596062214,54.46,59.06,80.86,92,100,12.82
1,1,Flinders,AU,1596062452,55.00,-34.58,150.86,87,0,7.00
2,2,Severo-Kuril'sk,RU,1596062286,55.76,50.68,156.12,81,98,13.40
3,3,Kapaa,US,1596062453,82.40,22.08,-159.32,69,20,18.34
4,4,Punta Arenas,CL,1596062453,39.20,-53.15,-70.92,80,99,13.87
...,...,...,...,...,...,...,...,...,...,...
566,566,Rocha,UY,1596062488,40.17,-34.48,-54.33,83,0,0.83
567,567,Jiaocheng,CN,1596062561,79.02,24.68,116.14,82,100,1.34
568,568,Port-Gentil,GA,1596062561,77.00,-0.72,8.78,83,75,14.32
569,569,Sri Aman,MY,1596062561,73.40,1.24,111.46,100,100,2.48


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig=gmaps.figure()
humidity = city_data["Humidity"].astype(float)
locations = city_data[["Latitude", "Longitude"]]
fig.add_layer( gmaps.heatmap_layer(locations, weights= humidity))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#ideal conditions for new df are humidity <80, wind speed < 5,cloudiness >70 Temperature >70 < 80
ideal_vaca_h = city_data.loc[city_data["Humidity"] < 80]
ideal_vaca_w =ideal_vaca_h.loc[ideal_vaca_h["Wind Speed"] < 5]
ideal_vaca_c =ideal_vaca_w.loc[ideal_vaca_w["Cloudiness"] > 70]
ideal_vaca_t =ideal_vaca_c.loc[ideal_vaca_c["Temperature"] > 70]
ideal_vaca =ideal_vaca_t.loc[ideal_vaca_t["Temperature"] < 80].dropna()
ideal_vaca

,Unnamed: 0,City Name,Country,Date,Temperature,Latitude,Longitude,Humidity,Cloudiness,Wind Speed
84,84,Itapirapuã,BR,1596062468,77.04,-15.82,-50.61,41,89,1.45
122,122,Padang,ID,1596062207,77.31,-0.95,100.35,73,99,4.36
192,192,Kurush,RU,1596062488,73.40,41.28,47.83,78,90,2.24
342,342,Camopi,GF,1596062516,78.10,3.17,-52.32,74,94,3.13
365,365,Rome,US,1596062427,73.40,43.21,-75.46,78,75,4.70
472,472,Meulaboh,ID,1596062543,77.54,4.14,96.13,77,95,4.50
506,506,High Prairie,CA,1596062525,75.92,55.43,-116.49,55,100,2.35


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
for index, row in ideal_vaca.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        ideal_vaca.loc[index, "Hotel Name"] = jsn['results'][0]['name']
    except:
        print("Missing field/result ... skipping")
ideal_vaca.dropna()
ideal_vaca.head()

Missing field/result ... skipping
Missing field/result ... skipping


,Unnamed: 0,City Name,Country,Date,Temperature,Latitude,Longitude,Humidity,Cloudiness,Wind Speed,Hotel Name
84,84,Itapirapuã,BR,1596062468,77.04,-15.82,-50.61,41,89,1.45,Hotel Nossa Senhora Aparecida
122,122,Padang,ID,1596062207,77.31,-0.95,100.35,73,99,4.36,Mercure Padang
192,192,Kurush,RU,1596062488,73.40,41.28,47.83,78,90,2.24,
342,342,Camopi,GF,1596062516,78.10,3.17,-52.32,74,94,3.13,
365,365,Rome,US,1596062427,73.40,43.21,-75.46,78,75,4.70,Wingate By Wyndham Rome


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_vaca.iterrows()]
locations = ideal_vaca[["Latitude", "Longitude"]]

In [25]:
# # Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))